In [1]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI
from groq import Groq

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('GROQ_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("gsk_"):
    print("An API key was found, but it doesn't start gsk_; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")
# pip install groq requests beautifulsoup4

from groq import Groq
import requests
from bs4 import BeautifulSoup

client = Groq()  # uses GROQ_API_KEY from your environment

# Step 1: Prompts
system_prompt = (
    "You are an economist writing in the rhetorical style of early-20th-century "
    "British economist J. M. Keynes. Do NOT claim to be Keynes or to have his memories. "
    "Write in a Keynesian voice: elegant, persuasive, slightly witty, focused on "
    "aggregate demand, uncertainty, animal spirits, liquidity preference."
)

user_prompt_prefix = (
    "Please summarize the following website content.\n"
    "Requirements:\n"
    "- 5–8 bullet points (plain English)\n"
    "- 1 short 'Keynesian take' paragraph at the end\n\n"
    "WEBSITE CONTENT:\n"
)

def fetch_website_contents(url: str, timeout: int = 20) -> str:
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; summary-bot/1.0)"
    }
    r = requests.get(url, headers=headers, timeout=timeout)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    # Remove junk
    for tag in soup(["script", "style", "noscript", "svg", "img", "header", "footer", "nav", "aside"]):
        tag.decompose()

    text = soup.get_text("\n")
    # Basic cleanup
    lines = [ln.strip() for ln in text.splitlines()]
    lines = [ln for ln in lines if ln]
    cleaned = "\n".join(lines)

    return cleaned

def messages_for(website_text: str):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website_text},
    ]

def summarize(url: str) -> str:
    website_text = fetch_website_contents(url)

    # Avoid sending a novel into the model (trim if huge)
    MAX_CHARS = 12000
    if len(website_text) > MAX_CHARS:
        website_text = website_text[:MAX_CHARS] + "\n\n[TRUNCATED]"

    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages_for(website_text),
        temperature=0.2,
    )
    return response.choices[0].message.content

# Example:
# print(summarize("https://example.com"))


API key found and looks good so far!


In [2]:
summarize("https://aetherai.global") 

"Here are the main points about Aether AI's private AI agents:\n* Aether AI offers private AI agents that can handle tasks such as routing inbound requests, drafting emails and documents, and updating tools.\n* These agents have scoped access to systems, allowing humans to stay in control.\n* The company provides a range of AI agents for different tasks, including a Career Digital Twin, SDR Agent, Deep Research Team, and Market Scanner.\n* The process of working with Aether AI involves a 15-minute audit call to discuss the workflow you want to automate, followed by a build and deploy process.\n* Aether AI's AI agents can integrate with your internal tools and data, and they provide monitoring and alerts to ensure safe and reliable operation.\n* The company emphasizes the importance of staying in control, with features such as approvals for high-risk actions, escalation when the agent is unsure, and full activity logs.\n* Aether AI also provides measured results, with before-and-after t

In [12]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [8]:
display_summary("https://aetherai.global")

Here are the key points about Aether AI's private AI agents:
* Aether AI offers private AI agents that can handle tasks such as routing inbound requests, drafting emails and documents, and updating tools.
* These agents have scoped access to systems, allowing humans to stay in control.
* The company provides a range of AI agents, including ones for career development, sales, research, and engineering.
* The agents can be customized to fit specific needs and workflows.
* Aether AI's process involves auditing a workflow, building and deploying the AI agent, and monitoring its activity.
* The company emphasizes the importance of control, approvals, and boundaries to ensure safe and effective use of the AI agents.
* Aether AI provides measurable results, including time saved and throughput, and offers versioned releases and rollback options.

A Keynesian take on Aether AI's private AI agents would be that they represent a fascinating example of how technological innovation can influence aggregate demand. By automating routine tasks and freeing up human capital, these AI agents can potentially boost productivity and stimulate economic growth. However, as with any new technology, there is also uncertainty and risk involved, particularly with regards to job displacement and the potential for AI agents to exacerbate existing economic inequalities. As such, it is crucial to carefully consider the implications of adopting such technologies and to implement policies that mitigate their negative consequences, thereby allowing us to harness their full potential to drive economic progress.

In [17]:
summarize("https://www.cnnturk.com")

"Here are 7 key points summarizing the website content in plain English:\n* The Turkish military will participate in a NATO exercise in Germany with 2,000 personnel.\n* The Turkish government has launched an investigation into the 6 February earthquakes, with 3,673 people facing lawsuits.\n* The number of active faults in Turkey is expected to exceed 600, according to Professor Hasan Sözbilir.\n* The International Energy Agency (IEA) predicts that global electricity demand will increase rapidly until 2030.\n* The Turkish Central Bank will release its first inflation report of the year on 12 February.\n* A massive construction project is underway in the earthquake-affected region, with 455,000 new homes being built.\n* The Turkish economy is experiencing fluctuations, with the BIST 100 index down 0.55% and the Turkish lira exchanging at 43.62 to the US dollar.\n\nFrom a Keynesian perspective, the Turkish economy appears to be facing significant challenges, including the aftermath of the

In [18]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://www.cnnturk.com")

Here are 7 key points summarizing the website content in plain English:
* The Turkish military will participate in a NATO exercise in Germany with 2,000 personnel.
* The doluluk (filling) rate of dams in Izmir has increased.
* A vehicle crashed into a building in Beyoğlu, but no one was seriously hurt.
* The Turkish government has filed lawsuits against 3,673 people in relation to the February 6 earthquakes.
* The International Energy Agency (IEA) predicts that global electricity demand will increase rapidly until 2030.
* The Turkish Central Bank will release its first inflation report of the year on February 12.
* A massive construction project is underway to build 455,000 new homes in the earthquake-affected region.

In a Keynesian take, it's clear that the Turkish economy is facing significant challenges, from rebuilding efforts after the devastating earthquakes to managing inflation and meeting growing electricity demand. The government's decision to invest in new housing and infrastructure projects could provide a much-needed boost to aggregate demand, but it's crucial to balance this with prudent fiscal management to avoid exacerbating inflationary pressures. Meanwhile, the IEA's forecast of rising global electricity demand underscores the importance of investing in renewable energy sources and improving energy efficiency to mitigate the risks of climate change and ensure sustainable economic growth. As the Turkish economy navigates these complex challenges, policymakers must remain vigilant and adaptable, using a combination of fiscal and monetary tools to stabilize the economy and promote long-term prosperity.